Preprocessing Data (Milestone 1)

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
def clean(fname):

    # Separate file into books with all metadata
    startBook = "<book"
    books = []
    with open(fname) as openFile:
        for line in openFile:
            if line[:len(startBook)] == startBook:
                books.append(line)
            else:
                books[-1] += line
    openFile.close()

    books = np.array(books)

    # Collect synopses and all genre tags for each book
    # X_data found here (synopses)
    synopses = []
    genres = []

    startBody = "<body>"
    endBody = "</body>"

    startTopic = "<topics>"
    endTopic = "</topics>"

    for book in books:
        start = book.index(startBody) + len(startBody)
        end = book.index(endBody)
        synopses.append(book[start:end])

        start = book.index(startTopic) + len(startTopic)
        end = book.index(endTopic)
        genres.append(book[start:end])

    X_data = np.array(synopses)


    # Create matrix of genre tags to book
    # Y_data found here
    start = ">"
    end = "</"

    Y_data = np.zeros((len(books), len(allGenres)))

    for i in range(len(books[:200])):
        entry = genres[i]
        while entry.find(end) != -1:
            s = entry.index(start) + 1
            e = entry.index(end)
            genreTag = entry[s:e]
            Y_data[i][allGenres.index(genreTag)] = 1
            entry = entry[e+5:]

    return (X_data, Y_data)




In [6]:
# Create a list of all genre tags
allGenres = []
with open("../data/hierarchy.txt") as hierarchyFile:
    for line in hierarchyFile:
        tab = line.find("\t")
        g1 = line[:tab]
        g2 = line[tab+1:len(line)-1]
        if allGenres.count(g1) == 0:
            allGenres.append(g1)
        if allGenres.count(g2) == 0:
            allGenres.append(g2)

print(allGenres[:4])

['Biography & Memoir', 'Arts & Entertainment Biographies & Memoirs', 'Political Figure Biographies & Memoirs', 'Historical Figure Biographies & Memoirs']


In [14]:
X_train, Y_train = clean("../data/BlurbGenreCollection_EN_train.txt")
X_test, Y_test = clean("../data/BlurbGenreCollection_EN_test.txt")
X_valid, Y_valid = clean("../data/BlurbGenreCollection_EN_dev.txt")




print(X_train[0])
print(Y_train[0])

print(X_test[0])
print(Y_test[0])

print(X_valid[0])
print(Y_valid[0])

Monday’s Crosswords Do with EaseTuesday’s Crosswords Not a BreezeWednesday’s Crosswords Harder StillThursday’s Crosswords Take Real SkillFriday’s Crosswords — You’ve Come This Far…Saturday’s Crosswords — You’re a Star!For millions of people, the New York Times crossword puzzles are as essential to each day as the first cup of coffee in the morning. Now, for the first time ever, these premier puzzles are available in six clever installments. With each day of the week, the puzzles increase gradually in skill level; Monday’s the easiest, but Saturday’s sure to challenge! Push your mental muscles a little harder each day with America’s favorite sophisticated — and fun — pastime: the New York Times crossword puzzles!The legendary Eugene T. Maleska was crossword editor of The New York Times from 1977 to 1993.
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [8]:
#X_trainDS = tf.data.Dataset.from_tensor_slices(X_train)
#X_validDS = tf.data.Dataset.from_tensor_slices(X_valid)
#X_testDS = tf.data.Dataset.from_tensor_slices(X_test)

In [9]:
from tensorflow.keras.layers import TextVectorization
import re
import string
AUTOTUNE = tf.data.AUTOTUNE


In [15]:
# Create Word2Vec word embeddings for each synopsis
# Start by indexing all words that appear into a vocuabulary list

# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 631 # approximately the largest synopsis we have

# Use the TextVectorization layer to normalize, split, and map strings to
# integers. Set output_sequence_length length to pad all samples to same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)



    

In [16]:
# Create global vocabulary list based off of train data
vectorize_layer.adapt(X_train)
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'and', 'of', 'a', 'to', 'in', 'is', 'for', 'with', 'that', 'her', 'his', 'as', 'on', 'from', 'this', 'an', 'by']


In [35]:
word2index = tf.keras.models.Sequential()
word2index.add(tf.keras.Input(shape=(1,), dtype=tf.string))
word2index.add(vectorize_layer)

In [36]:
X_train = word2index.predict(X_train)
X_test = word2index.predict(X_test)
X_valid = word2index.predict(X_valid)

In [44]:
for i in range(20, 40):
    print(inverse_vocab[X_train[0][i]])

this
[UNK]
[UNK]
—
you’re
a
[UNK]
millions
of
people
the
new
york
times
crossword
puzzles
are
as
essential
to


In [22]:
vocab_2_index = dict((c,i) for i, c in enumerate(inverse_vocab)) 
index_2_vocab = dict((i,c) for i, c in enumerate(inverse_vocab))

In [ ]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

Create the Model and Training! (Milestone 2)

In [173]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.layers import Flatten, Conv1D, MaxPool1D
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import BinaryCrossentropy


In [174]:
model = Sequential()

# DEFINE seq_length as length of longest blurb...maybe?
nb_filters=16
filter_size=3
seq_length = 631
model.add(Conv1D(filters=nb_filters, kernel_size=filter_size, activation='sigmoid', 
                 kernel_initializer='glorot_normal', input_shape=(seq_length, 1)))
model.add(MaxPool1D()) 
model.add(Conv1D(filters=nb_filters, kernel_size=filter_size, activation='sigmoid', 
                 kernel_initializer='glorot_normal'))
model.add(Conv1D(filters=nb_filters, kernel_size=filter_size, activation='sigmoid', 
                 kernel_initializer='glorot_normal'))
model.add(MaxPool1D()) 
model.add(Conv1D(filters=nb_filters, kernel_size=filter_size, activation='sigmoid', 
                 kernel_initializer='glorot_normal'))
model.add(MaxPool1D()) 
model.add(Flatten())
model.add(Dense(1, activation='linear'))

In [175]:
model.compile(loss='BinaryCrossentropy', optimizer='adam', metrics=['accuracy'])

In [176]:
es = EarlyStopping(monitor='val_loss', 
                   patience=50, verbose=1,
                   restore_best_weights=True)



In [177]:
import tensorflow_datasets as tfds

In [179]:
x_train = []
for x in tfds.as_numpy(X_train):
    x_train.append(x)





TypeError: Arguments to as_numpy must be tf.Tensors or tf.data.Datasets. Got: <class 'numpy.ndarray'>.

In [171]:
# what is going on?
# Error Message: 
"""/Users/lanearohan/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: 
VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of 
lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. 
If you meant to do this, you must specify 'dtype=object' when creating the ndarray
#Entry point for launching an IPython kernel."""

X_train = np.array(x_train)

/Users/lanearohan/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [ ]:

#Y_train.size()

NameError: name 'count' is not defined

In [ ]:
# size issues between X_train and Y_train ... ????????

model.fit(x_train, Y_train, batch_size=32, validation_split=0.2, callbacks=[es], epochs=100)

ValueError: Data cardinality is ambiguous:
  x sizes: 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 5

In [ ]:
preds = model.predict(X_test)

2021-11-18 21:26:10.570553: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
